In [5]:
# Run me first to import and setup everything
import time
import csv  
from myapikey import APIkey
from alpha_vantage.timeseries import TimeSeries
import pickle
import numpy as np
import tensorflow as tf

def add_classification(stocks):
    new_stocks = []
    last_price = stocks[len(stocks) - 1][4]
    new_stocks.append(np.insert(stocks[len(stocks) - 1], 5, 1)) # up
    for s in reversed(list(range(len(stocks) - 1))):
        if (stocks[s][4] - last_price) > 0:
            new_stocks.append(np.insert(stocks[s], 5, 1)) # up
        else:
            new_stocks.append(np.insert(stocks[s], 5, 0)) # down
        last_price = stocks[s][4]
    return new_stocks

def split_sample(data):
    sample = []
    for start in range(0, len(data) - 50, 50):
        sample.append(data[start:start+50])
    return sample

def prepare(group_of_stocks):
    prepared_data_set = []
    for stock in group_of_stocks:
        prepared_data_set.append(split_sample(add_classification(stock[0].to_numpy())))
    return prepared_data_set

def prep_part_two(group_of_stocks):
    new_set = []
    for stock in group_of_stocks:
        for collection in stock:
            classification = collection[0][5]
            training_data = collection[1:]
            new_set.append([training_data, classification])
    return new_set

# def get_intraday(stock):
#     ts = TimeSeries(key=APIkey, output_format='pandas', indexing_type='integer')
#     return ts.get_intraday(symbol=stock, interval='1min', outputsize='full')

def get_daily(stock):
    ts = TimeSeries(key=APIkey, output_format='pandas')
    data, meta_data = ts.get_daily(symbol=stock, outputsize='full')
    return data, meta_data

def open_csv(path_to_csv):
    data = []
    with open(path_to_csv) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            if row[0] != '':
                data.append(row)
    return data

def batch_get_daily(stock_list):
    print("total (", len(stock_list), ")")
    big_counter = 0
    count = 0
    data = []
    for stock in stock_list:
        if count == 5:
            time.sleep(65) # 65 seconds just in case the timing on the server or here isn't perfect
            count = 0
        data.append(get_daily(stock))
        big_counter += 1
        print(stock, ":", big_counter, end=" ")
        count += 1
    return data


In [ ]:
##########################################
#######                           ########
#######    WOULD LIKE TO HAVES    ########
#######                           ########
##########################################

print("would like to haves")


In [ ]:
# Save data as CSV



In [8]:
# Wait for API (free) access limit
import time

def batch_get_daily(stock_list):
    print("total (", len(stock_list), ")")
    big_counter = 0
    count = 0
    data = []
    for stock in stock_list:
        if count == 5:
            time.sleep(65) # 65 seconds just in case the timing on the server or here isn't perfect
            count = 0
        try:
            data.append(get_daily(stock))
            big_counter += 1
        except:
            print(stock, " failed ", big_counter, end=" ")
        else:
            print(stock, ":", big_counter, end=" ")
        count += 1
    return data


In [ ]:
# Save results of Training
# https://www.tensorflow.org/guide/keras#entire_model
model.save('verdurouMKI.h5')


In [ ]:
# Load Results from Previous Training
loaded_model = tf.keras.models.load_model('verdurouMKI.h5')
print(loaded_model)

In [ ]:
# Scrape HTML


In [ ]:
##########################################
############                  ############
############    MUST HAVES    ############
############                  ############
##########################################

print("Must Haves")


In [ ]:
# Read CSV
# This will read in the CSVs that come from the alpha vantage website, where each CSV
# is from a separate stock symbol
import csv  

def open_csv(path_to_csv):
    data = []
    with open(path_to_csv) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            if row[0] != '':
                data.append(row)
    return data


In [ ]:
# Access API
from myapikey import APIkey
from alpha_vantage.timeseries import TimeSeries


In [ ]:
# must provide an api key (they are free from the alpha vantage website
# this determines how much data you can get in how much time.
def get_intraday(stock):
    ts = TimeSeries(key=APIkey, output_format='pandas', indexing_type='integer')
    return ts.get_intraday(symbol=stock, interval='1min', outputsize='full')

# provide an api key to access the data
def get_daily(stock):
    ts = TimeSeries(key=APIkey, output_format='pandas')
    data, meta_data = ts.get_daily(symbol=stock, outputsize='full')
    return data, meta_data


In [4]:
# Select stocks based on ticker symbol
stock_names = ["A", "AAL", "AAP", "AAPEX", "AAPH", "AAPIX", "AAPJ", "AAPL", "ABBV", "ABEV", "ACB", "ACN", "ADBE", "ADIL", "ADM", "ADPT", "ADS", "AEE", "AEP", "AES", "AETI", "AFL", "AGNC", "AIG", "AIV", "AIZ", "AJG", "AKAM", "ALB", "ALGN", "ALK", "ALKS", "ALL", "ALLE", "ALLY", "ALXN", "AMAT", "AMCR", "AME", "AMG", "AMGN", "AMP", "AMRH", "AMRN", "AMT", "AMZN", "ANET", "ANSS", "ANTM", "ANY", "AON", "AOS", "APA", "APC", "APD", "APH", "APTV", "AQST", "ARCE", "ARE", "ARLO", "ARNC", "ARVN", "ATO", "ATVI", "AUY", "AVB", "AVGR", "AVTR", "AVY", "AXP", "AXSM", "AZO", "BA", "BABA", "BAC", "BAX", "BBY", "BEN", "BHGE", "BKNG", "BLL", "BLRX", "BMY", "BRK.B", "BSQR", "BSX", "BTAI", "BWA", "CAG", "CAH", "CAT", "CB", "CBRE", "CCI", "CDNS", "CELG", "CERN", "CGA", "CHRW", "CI", "CINF", "CL", "CLIR", "CLX", "CMA", "CMCSA", "CME", "CMG", "CMI", "CNC", "CNP", "COF", "COG", "COO", "COST", "COTY", "COUP", "CPB", "CPRI", "CPSH", "CSX", "CTAS", "CTK", "CTL", "CTRV", "CTSH", "CTVA", "CTXS", "CVX", "CX", "CXO", "D", "DAL", "DD", "DE", "DFS", "DHR", "DISCA", "DISCK", "DISH", "DLR", "DLTR", "DRE", "DRI", "DUK", "DVA", "DVN", "DXC", "EA", "EBAY", "ECA", "ECL", "ED", "EFX", "EMN", "EMR", "EOG", "EPD", "EQIX", "ES", "ESS", "ET", "ETN", "ETR", "EVRG", "EW", "EXC", "EXPD", "F", "FANG", "FARO", "FAST", "FB", "FBHS", "FCX", "FDC", "FDS", "FDX", "FE", "FFIV", "FIS", "FITB", "FL", "FLIR", "FLKS", "FLS", "FLT", "FMC", "FOX", "FOXA", "FRAN", "FRC", "FRT", "FTI", "FTNT", "FTV", "GE", "GFI", "GGB", "GILD", "GIS", "GLW", "GM", "GNMX", "GOLD", "GOOG", "GOOGL", "GPC", "GPK", "GPS", "GRMN", "GWW", "HAL", "HAS", "HBAN", "HBI", "HCA", "HCP", "HD", "HFC", "HII", "HLT", "HMC", "HOG", "HOV", "HP", "HPE", "HPQ", "HSGX", "HST", "HSY", "HUM", "IBM", "ICE", "IDXX", "IFF", "IGLD", "ILMN", "INCY", "INFO", "INFY", "INOD", "INTC", "IP", "IPG", "IPGP", "IR", "IRM", "ISRG", "IT", "ITUB", "ITW", "IVZ", "JD", "JEC", "JEF", "JKHY", "JNJ", "JNPR", "JPM", "JT", "JWN", "K", "KEY", "KEYS", "KGC", "KHC", "KIM", "KLAC", "KMB", "KMI", "KMX", "KO", "KOSS", "KR", "KSS", "KSU", "LEG", "LEN", "LHX", "LIN", "LKQ", "LMT", "LNT", "LOW", "LPTH", "LRCX", "LUV", "LYB", "M", "MA", "MAA", "MAC", "MAS", "MCHP", "MCK", "MCO", "MDLZ", "MDT", "MET", "MHK", "MKC", "MKTX", "MLM", "MMC", "MMM", "MNST", "MO", "MOS", "MPC", "MRK", "MRVL", "MS", "MSCI", "MTB", "MU", "MXIM", "MYSZ", "NBL", "NCLH", "NDAQ", "NEE", "NFLX", "NI", "NIO", "NKE", "NKTR", "NOC", "NOK", "NOV", "NRG", "NTAP", "NTNX", "NUE", "NVDA", "NVFY", "NWL", "NWS", "NWSA", "NYCB", "O", "OKE", "OMC", "ORCL", "ORLY", "OSS", "OXBR", "OXY", "PAYX", "PBCT", "PDD", "PEG", "PEP", "PFE", "PG", "PGR", "PH", "PHM", "PIR", "PKI", "PLAN", "PNC", "PNR", "PPG", "PPL", "PRGO", "PRU", "PSA", "PSX", "PVH", "PXD", "PYPL", "QBAK", "QRVO", "QUIK", "RBCN", "RCL", "RE", "REG", "REGN", "RHI", "RHT", "RIG", "RL", "RMD", "ROK", "ROKU", "ROL", "ROP", "ROST", "S", "SAN", "SBAC", "SBUX", "SCHW", "SEE", "SESN", "SFUN", "SGMA", "SHW", "SIRI", "SJM", "SLB", "SLG", "SMAR", "SNAP", "SNPS", "SO", "SPGI", "SRE", "STT", "STX", "STZ", "SWK", "SWKS", "SYK", "SYMC", "SYY", "TAP", "TDG", "TEL", "TEVA", "TFX", "TGT", "TIF", "TJX", "TME", "TMK", "TMO", "TMUS", "TNK", "TOPS", "TRHC", "TRIL", "TRIP", "TROW", "TRQ", "TRV", "TSLA", "TSM", "TSN", "TSS", "TTWO", "TWMC", "TWTR", "TXT", "UAA", "UAL", "UBER", "UDR", "UHS", "ULTA", "UNM", "UNP", "UPL", "UPS", "URI", "USB", "V", "VALE", "VEON", "VFC", "VIAB", "VIPS", "VLO", "VMC", "VRSK", "VRTX", "VTVT", "VZ", "WAT", "WBA", "WCG", "WDC", "WELL", "WFC", "WHR", "WLTW", "WM", "WMB", "WMT", "WPX", "WRK", "WU", "WY", "WYNN", "XEC", "XEL", "XLNX", "XOM", "XRAY", "XRX", "XYL", "YUM", "ZBH", "ZION", "ZNGA", "ZTS"]
print(len(stock_names))

499


In [ ]:
# check if the stock is valid

# import requests
# import re
# import time

# target_string = "{\"GlobalQuote\":{}}"
# _bad_string = "{\"Note\": \"Thank you for using Alpha Vantage! Our standard API call frequency is 5 calls per minute and 500 calls per day. Please visit https://www.alphavantage.co/premium/ if you would like to target a higher API call frequency.\"}"
# bad_string = re.sub(r"\s", "", _bad_string)
# count = 0
# true_count = 0
# true_length = len(stock_names) / 5

# for stock in stock_names:
#     _result = requests.get("https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol=" + stock + "&apikey=SSAQZVKJHSN826NT").text
#     result = re.sub(r"\s", "", _result)

#     if (result == bad_string):
#         print("overtime")
#         time.sleep(60)
#         _result = requests.get("https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol=" + stock + "&apikey=SSAQZVKJHSN826NT").text
#         result = re.sub(r"\s", "", _result)
    
#     if (result == target_string):
#         print(stock, result)
    
#     count += 1
#     if (count == 5):
#         count = 0
#         true_count += 1
#         print("progress: ", true_count, "/", true_length)
#         time.sleep(61)


In [9]:
# Store Stock information as a data set
stock_data = batch_get_daily(stock_names)


total ( 499 )
A : 1 AAL : 2 AAP : 3 AAPEX : 4 AAPH : 5 AAPIX : 6 AAPJ : 7 AAPL : 8 ABBV : 9 ABEV : 10 ACB : 11 ACN : 12 ADBE : 13 ADIL : 14 ADM : 15 ADPT : 16 ADS : 17 AEE : 18 AEP : 19 AES : 20 AETI : 21 AFL : 22 AGNC : 23 AIG : 24 AIV : 25 AIZ : 26 AJG : 27 AKAM : 28 ALB : 29 ALGN : 30 ALK : 31 ALKS : 32 ALL : 33 ALLE : 34 ALLY : 35 ALXN : 36 AMAT : 37 AMCR : 38 AME : 39 AMG : 40 AMGN : 41 AMP : 42 AMRH : 43 AMRN : 44 AMT : 45 AMZN : 46 ANET : 47 ANSS : 48 ANTM : 49 ANY : 50 AON : 51 AOS : 52 APA : 53 APC : 54 APD : 55 APH : 56 APTV : 57 AQST : 58 ARCE : 59 ARE : 60 ARLO : 61 ARNC : 62 ARVN : 63 ATO : 64 ATVI : 65 AUY : 66 AVB : 67 AVGR : 68 AVTR : 69 AVY : 70 AXP : 71 AXSM : 72 AZO : 73 BA : 74 BABA : 75 BAC : 76 BAX : 77 BBY : 78 BEN : 79 BHGE : 80 BKNG : 81 BLL : 82 BLRX : 83 BMY : 84 BRK.B : 85 BSQR : 86 BSX : 87 BTAI : 88 BWA : 89 CAG : 90 CAH : 91 CAT : 92 CB : 93 CBRE : 94 CCI : 95 CDNS : 96 CELG : 97 CERN : 98 CGA : 99 CHRW : 100 CI : 101 CINF : 102 CL : 103 CLIR : 104 CLX : 

In [10]:
# print(stock_data[0][0])
import pickle


In [11]:
f = open('store_stocks_new.pckl', 'wb') # rename and remove '_new' to load it
pickle.dump(stock_data, f)
f.close()


In [ ]:
import pickle
f = open('store_stocks.pckl', 'rb')
stock_data = pickle.load(f)
f.close()

print(len(stock_data))


In [12]:
# Format dataset with classification (up/down)
import numpy as np

def add_classification(stocks):
    # up = 1, down = 0
    new_stocks = []
    # column 4 is the end of day price
    last_price = stocks[len(stocks) - 1][4]
    # the very last price will always be up. Perhaps
    # this makes my AI optimistic?
    new_stocks.append(np.insert(stocks[len(stocks) - 1], 5, 1)) # up
    for s in reversed(list(range(len(stocks) - 1))):
        if (stocks[s][4] - last_price) > 0:
            new_stocks.append(np.insert(stocks[s], 5, 1)) # up
        else:
            new_stocks.append(np.insert(stocks[s], 5, 0)) # down
        last_price = stocks[s][4]
    return new_stocks

# accepts an array of all data from one stock
# produces chunks to be made into tensors
# run this once on each stock and save the resulting array (of arrays)
def split_sample(data):
    sample = []
    for start in range(0, len(data) - 50, 50):
        sample.append(data[start:start+50])
    return sample

# takes the results of batch_get_daily and prepares everything
def prepare(group_of_stocks):
    prepared_data_set = []
    for stock in group_of_stocks:
        prepared_data_set.append(split_sample(add_classification(stock[0].to_numpy())))
    return prepared_data_set

def prep_part_two(group_of_stocks):
    new_set = []
    for stock in group_of_stocks:
        for collection in stock:
            classification = collection[0][5]
            training_data = collection[1:]
            new_set.append([training_data, classification])
    return new_set

# prepped_stock[n][m][0][5] = the classification!
# prepped_stock[n][m][X > 0] = the data sets!


In [13]:
# an example of how to use this data and functions
# print(split_sample(add_classification(stock_data[0][0].to_numpy()))[0])

# results of preparing the stocks
prepped_stocks = prepare(stock_data)


In [ ]:
# pickle the prepped stocks!
import pickle


In [ ]:
f = open('store_prepped_stocks_new.pckl', 'wb') # rename and remove '_new' to load it
pickle.dump(prepped_stocks, f)
f.close()


In [ ]:
f = open('store_prepped_stocks.pckl', 'rb')
prepped_stocks = pickle.load(f)
f.close()

print(len(prepped_stocks))

In [ ]:
# prepped stocks part 2
data_set = prep_part_two(prepped_stocks)

# now the data should be one array filled with pairs of 49 days worth of 
# data with the classification of the next day.

# TODO: so I need to split this into x_train, y_train and x_test, y_test

print(len(data_set))


In [ ]:
# save my work
import pickle


In [ ]:
f = open('store_data_set_new.pckl', 'wb') # rename and remove '_new' to load it
pickle.dump(data_set, f)
f.close()


In [ ]:
f = open('store_data_set.pckl', 'rb')
data_set = pickle.load(f)
f.close()


In [ ]:
# x_train, y_train, x_test, y_test
train_data = []
train_class = []
test_data = []
test_class = []

np.random.shuffle(data_set)

# print(data_set[:4])

for combo in data_set:
    train_data.append(combo[0])
    train_class.append(combo[1])

test_data = train_data[7707:9707]
test_class = train_class[7707:9707]
train_data = train_data[0:7706]
train_class = train_class[0:7706]

In [ ]:
# save my work
import pickle
# train_data = []
# train_class = []
# test_data = []
# test_class = []



In [ ]:
f = open('store_train_data_new.pckl', 'wb') # rename and remove '_new' to load it
pickle.dump(train_data, f)
f.close()

f = open('store_train_class_new.pckl', 'wb') # rename and remove '_new' to load it
pickle.dump(train_class, f)
f.close()

f = open('store_test_data_new.pckl', 'wb') # rename and remove '_new' to load it
pickle.dump(test_data, f)
f.close()

f = open('store_test_class_new.pckl', 'wb') # rename and remove '_new' to load it
pickle.dump(test_class, f)
f.close()


In [ ]:
f = open('store_train_data.pckl', 'rb')
train_data = pickle.load(f)
f.close()

f = open('store_train_class.pckl', 'rb')
train_class = pickle.load(f)
f.close()

f = open('store_test_data.pckl', 'rb')
test_data = pickle.load(f)
f.close()

f = open('store_test_class.pckl', 'rb')
test_class = pickle.load(f)
f.close()


In [ ]:
# (3 must-haves in one cell)
# Train
# Hidden Nodes
# Dropout
import tensorflow as tf

# example training
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(49, 6)),
    tf.keras.layers.Dense(512, activation=tf.nn.softmax),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(512, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(512, activation=tf.nn.relu),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(np.array(train_data), np.array(train_class), epochs=7)


In [ ]:
# Test
model.evaluate(np.array(test_data), np.array(test_class))


In [ ]:
# Real Life Prediction
def get_daily_compact(stock):
    ts = TimeSeries(key=APIkey, output_format='pandas')
    data, meta_data = ts.get_daily(symbol=stock, outputsize='compact') # this returns 100 datapoints
    return data

chosen_stock = "AAPL"
rlp_stock = get_daily_compact(chosen_stock)
rlp_data = split_sample(add_classification(rlp_stock.to_numpy()))[0]
rlp_data = [rlp_data[:49]]

prediction = model.predict_classes(np.array(rlp_data), batch_size=1, verbose=0)

In [ ]:
# Print results of prediction

# 1 = tomorrow will be up
# 0 = tomorrow will be down

if (prediction == 1):
    print(chosen_stock + " will go up tomorrow")
else:
    print(chosen_stock + " will go down tomorrow")